# **Initialisation**
Initialisation de la base de données spark

In [1]:
import os
import findspark

findspark.init()

import spark


In [2]:
import configparser

config = configparser.ConfigParser()

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.master("local").appName("Import").getOrCreate()

22/12/02 13:43:41 WARN Utils: Your hostname, marins-UX430UAR resolves to a loopback address: 127.0.1.1; using 147.250.227.154 instead (on interface wlp2s0)
22/12/02 13:43:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/02 13:43:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# **Extraction**
Premier regard sur la base de données 

In [4]:
df = spark.read.option("multiline","true").json("/home/marins/3A-ENSTA/Big data project/nvdcve-1.1-2022.json")
df = df.select(explode(col("CVE_Items")))
df = df.select("col.cve.CVE_data_meta.*","col.impact.baseMetricV3.cvssV3.*").drop("version").drop("vectorString")

json_files = os.listdir("/home/marins/3A-ENSTA/Big data project/CVE_Analysis/src/")
for json in json_files:
      if json_files != 'nvdcve-1.1-2022.json' :
            print("Importing", json,"...")
            df_temp = spark.read.option("multiline","true").json("/home/marins/3A-ENSTA/Big data project/CVE_Analysis/src/"+json)
            df_temp = df_temp.select(explode(col("CVE_Items")))
            df_temp = df_temp.select("col.cve.CVE_data_meta.*","col.impact.baseMetricV3.cvssV3.*").drop("version").drop("vectorString")
            df = df.union(df_temp)


print("Schema initial : ")
df.printSchema()


Importing nvdcve-1.1-2022.json ...


Importing nvdcve-1.1-2020.json ...


Importing nvdcve-1.1-2018.json ...


Importing nvdcve-1.1-2016.json ...


Importing nvdcve-1.1-2021.json ...


Importing nvdcve-1.1-2019.json ...


Importing nvdcve-1.1-2017.json ...


Schema initial : 
root
 |-- ASSIGNER: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- attackComplexity: string (nullable = true)
 |-- attackVector: string (nullable = true)
 |-- availabilityImpact: string (nullable = true)
 |-- baseScore: double (nullable = true)
 |-- baseSeverity: string (nullable = true)
 |-- confidentialityImpact: string (nullable = true)
 |-- integrityImpact: string (nullable = true)
 |-- privilegesRequired: string (nullable = true)
 |-- scope: string (nullable = true)
 |-- userInteraction: string (nullable = true)



Extraction des données pertinents via select(). 

In [5]:
print("Columns : ",df.columns)
#df.printSchema()

Columns :  ['ASSIGNER', 'ID', 'attackComplexity', 'attackVector', 'availabilityImpact', 'baseScore', 'baseSeverity', 'confidentialityImpact', 'integrityImpact', 'privilegesRequired', 'scope', 'userInteraction']


Affichage des premières lignes et du nombre de samples à disposition pour se faire une idée.

In [6]:
lines_count = df.count()
print("Number of available rows : ", lines_count)
df.show(5)

Number of available rows :  136047
22/12/02 13:44:15 WARN DAGScheduler: Broadcasting large task binary with size 1428.0 KiB


+--------------------+-------------+----------------+------------+------------------+---------+------------+---------------------+---------------+------------------+---------+---------------+
|            ASSIGNER|           ID|attackComplexity|attackVector|availabilityImpact|baseScore|baseSeverity|confidentialityImpact|integrityImpact|privilegesRequired|    scope|userInteraction|
+--------------------+-------------+----------------+------------+------------------+---------+------------+---------------------+---------------+------------------+---------+---------------+
|    secure@intel.com|CVE-2022-0001|             LOW|       LOCAL|              NONE|      6.5|      MEDIUM|                 HIGH|           NONE|               LOW|  CHANGED|           NONE|
|    secure@intel.com|CVE-2022-0002|             LOW|       LOCAL|              NONE|      6.5|      MEDIUM|                 HIGH|           NONE|               LOW|  CHANGED|           NONE|
|    secure@intel.com|CVE-2022-0004|    

Essai de la fonction tail pour récupérer des données. Pas idéal. La fonction "collect()" semble bien plus adaptée.

In [7]:
print(df.tail(1)) 

22/12/02 13:44:18 WARN DAGScheduler: Broadcasting large task binary with size 1424.8 KiB


[Row(ASSIGNER='cve@mitre.org', ID='CVE-2017-9999', attackComplexity=None, attackVector=None, availabilityImpact=None, baseScore=None, baseSeverity=None, confidentialityImpact=None, integrityImpact=None, privilegesRequired=None, scope=None, userInteraction=None)]


# Machine Learning Prediction
Tentative de prédiction du baseScore d'une CVE via les données cvssV3 ingurgitées.


In [8]:
import pandas as pd
pandasDF = df.toPandas()
print(pandasDF.columns)

print(pandasDF.shape[0])

pandasDF = pandasDF[pandasDF['baseScore'].notna()]

print(pandasDF.shape[0])


pandasDF.head(3)

22/12/02 13:44:21 WARN DAGScheduler: Broadcasting large task binary with size 1424.9 KiB


Index(['ASSIGNER', 'ID', 'attackComplexity', 'attackVector',
       'availabilityImpact', 'baseScore', 'baseSeverity',
       'confidentialityImpact', 'integrityImpact', 'privilegesRequired',
       'scope', 'userInteraction'],
      dtype='object')
136047
127498


,ASSIGNER,ID,attackComplexity,attackVector,availabilityImpact,baseScore,baseSeverity,confidentialityImpact,integrityImpact,privilegesRequired,scope,userInteraction
0,secure@intel.com,CVE-2022-0001,LOW,LOCAL,NONE,6.5,MEDIUM,HIGH,NONE,LOW,CHANGED,NONE
1,secure@intel.com,CVE-2022-0002,LOW,LOCAL,NONE,6.5,MEDIUM,HIGH,NONE,LOW,CHANGED,NONE
2,secure@intel.com,CVE-2022-0004,LOW,PHYSICAL,HIGH,6.8,MEDIUM,HIGH,HIGH,NONE,UNCHANGED,NONE


In [9]:
target_column = "baseScore"
target = pandasDF[target_column]

categorical_columns = ["attackComplexity",
                       "attackVector",
                       "availabilityImpact",
                       "confidentialityImpact",
                       "integrityImpact",
                       "privilegesRequired",
                       "scope",
                       "userInteraction"]

data = pandasDF[categorical_columns]


Affichage des types des différentes colonnes

In [10]:
target.dtypes

dtype('float64')

In [11]:
data.dtypes

attackComplexity         object
attackVector             object
availabilityImpact       object
confidentialityImpact    object
integrityImpact          object
privilegesRequired       object
scope                    object
userInteraction          object
dtype: object

In [12]:
import numpy as np
#pd.set_option("display.max_rows", None)

target.value_counts().sort_index()

len(np.where(np.isnan(target))[0])


0

In [13]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.linear_model import LogisticRegression, SGDClassifier, Lasso
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeRegressor # very good
from sklearn.linear_model import LinearRegression # very good
from sklearn.ensemble import HistGradientBoostingClassifier

from sklearn.pipeline import make_pipeline

# Onehot ==> Linéaire
# Ordinal = Treebased

model = make_pipeline(OneHotEncoder(sparse=False),LinearRegression())
model

Pipeline(steps=[('onehotencoder', OneHotEncoder(sparse=False)),
                ('linearregression', LinearRegression())])

In [14]:
from sklearn.model_selection import cross_validate

cv_results = cross_validate(model, data, target, cv = 5)


In [15]:
cv_results["test_score"].mean()

0.9570179799045553

# Plotting 

In [27]:
import numpy as np
import plotly.express as px
import numpy as np

plotready_PandasDF = pandasDF[categorical_columns+[target_column]]
plotready_PandasDF = plotready_PandasDF.sort_values(by=['baseScore'], ascending=False)
plotready_PandasDF = plotready_PandasDF.reset_index()
plotready_PandasDF['baseScore'] = plotready_PandasDF["baseScore"].apply(np.ceil)
#plotready_PandasDF = plotready_PandasDF["attackVector", "attackComplexity", "privilegesRequired", "scope"]


fig = px.parallel_categories(
    plotready_PandasDF, 
    labels={"baseScore": "Score CVSSV3",
                "attackComplexity": "Complexité", "attackVector": "Vecteur d'attaque",
                "availabilityImpact": "Impact : Disponibilité", "confidentialityImpact": "Impact : Confidentialité",
                "integrityImpact": "Impact : Intégrité" , "privilegesRequired": "Privilèges nécessaires", "userInteraction" : "Interaction utilisateur",
                "baseSeverity" : "Sévérité"},
    color="baseScore",
    color_continuous_scale=px.colors.sequential.Viridis,
)
fig.show()

KeyError: ('attackVector', 'attackComplexity', 'privilegesRequired', 'scope')